In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# อ่านไฟล์ข้อมูล
nonPhishingDf = pd.read_csv('Non-Phishing.csv')
phishingDf = pd.read_csv('Phishing.csv')

In [4]:
# จัดการคอลัมน์ให้สอดคล้องกัน
# เรียงคอลัมน์ให้เหมือนกัน: sender, receiver, date, subject, body, urls, label
nonPhishingDf = nonPhishingDf[['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label']]
phishingDf = phishingDf[['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label']]

In [5]:
# ทำความสะอาดข้อมูล โดยลบแถวที่มีค่าว่างในคอลัมน์ที่สำคัญ
allEmails = pd.concat([nonPhishingDf, phishingDf], ignore_index=True)
allEmails.dropna(subset=['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label'], inplace=True)

In [6]:
# แบ่งข้อมูลเป็นฟีเจอร์และเลเบล
X = allEmails.drop('label', axis=1)
y = allEmails['label']

In [7]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ โดยให้มีอัตราส่วนของอีเมลธรรมดาและอีเมล phishing ใกล้เคียงกัน
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# บันทึกชุดข้อมูลฝึกและทดสอบลงไฟล์
X_train.to_csv('TrainingSet.csv', index=False)
y_train.to_csv('TrainingLabels.csv', index=False)
X_test.to_csv('TestSet.csv', index=False)
y_test.to_csv('TestLabels.csv', index=False)